In [5]:
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
matplotlib.use('Qt4Agg')
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

def trimf(x, abc):
    a, b, c = abc
    y = np.zeros(len(x))
    left_side_eq = (1/(b-a))*(x-a)
    right_side_eq = (1/(c-b))*(-x+c)
    y = np.minimum(left_side_eq, right_side_eq)
    y = np.maximum(y,0)
    return y

def inference_mandani_2_premissas_and(p1, ux1, p2, ux2, uy):
    antecedente1_max = np.max(np.minimum(p1[:], ux1[:]))
    antecedente2_max = np.max(np.minimum(p2[:], ux2[:]))
    antecedente_max = min(antecedente1_max,antecedente2_max)
    q = np.minimum(antecedente_max, uy[:])
    return q

def inference_mandani(p1, ux1, uy):
    antecedente_max = np.max(np.minimum(p1[:], ux1[:]))
    q = np.minimum(antecedente_max, uy[:])
    return q

def defuzz(x, fuz):

    momento_total = 0.0
    area_total = 0.0

    for i in range(1, len(x)):
        x1 = x[i - 1]
        x2 = x[i]
        y1 = fuz[i - 1]
        y2 = fuz[i]


        if not(y1 == y2 == 0.0 or x1 == x2):
            if y1 == y2:  # Cálculo para retângulo
                moment = 0.5 * (x1 + x2)
                area = (x2 - x1) * y1
            elif y1 == 0.0 and y2 != 0.0:  # Triângulo
                moment = 2.0 / 3.0 * (x2-x1) + x1
                area = 0.5 * (x2 - x1) * y2
            elif y2 == 0.0 and y1 != 0.0:  # Triângulo
                moment = 1.0 / 3.0 * (x2 - x1) + x1
                area = 0.5 * (x2 - x1) * y1
            else:
                moment = (2.0 / 3.0 * (x2-x1) * (y2 + 0.5*y1)) / (y1+y2) + x1
                area = 0.5 * (x2 - x1) * (y1 + y2)

            momento_total += moment * area
            area_total += area

    return momento_total / np.fmax(area_total,
                                     np.finfo(float).eps).astype(float)

def trapmf(x, abcd):
    y = np.ones(len(x))
    a, b, c, d = abcd
    idx = np.nonzero(x < b)[0]
    y[idx] = trimf(x[idx], np.r_[a, b, b])

    idx = np.nonzero(x > c)[0]
    y[idx] = trimf(x[idx], np.r_[c, c, d])

    idx = np.nonzero(x < a)[0]
    y[idx] = np.zeros(len(idx))

    idx = np.nonzero(x > d)[0]
    y[idx] = np.zeros(len(idx))

    return y

def inference_PROD_2_premissas_and(p1, ux1, p2, ux2, uy):
    antecedente1_max = np.max(np.multiply(p1[:], ux1[:]))
    antecedente2_max = np.max(np.multiply(p2[:], ux2[:]))
    antecedente_max = min(antecedente1_max,antecedente2_max)
    q = np.multiply(antecedente_max, uy[:])
    return q

def inference_PROD(p1, ux1, uy):
    antecedente_max = np.max(np.multiply(p1[:], ux1[:]))
    q = np.multiply(antecedente_max, uy[:])
    return q

In [6]:
x = np.arange(0, 10.1, 0.1)
y = np.arange(0, 26, 1)

us_ruim = trimf(x, [-5, 0, 5])
us_acpt = trimf(x, [0, 5, 10])
us_bom = trimf(x, [5, 10, 15])

ur_ruim = trimf(x, [-5, 0, 5])
ur_acpt = trimf(x, [2, 5, 8])
ur_bom = trimf(x, [6, 10, 14])

yg_ruim = trimf(y, [-12.5, 0, 12.5])
yg_acpt = trimf(y, [0, 12.5, 25])
yg_bom = trimf(y, [12.5, 25, 37.5])

resultado = np.zeros((101,101))
for elem in x:
    par1 = np.zeros(x.shape[0])
    par1[np.where(x==elem)] = 1
    for elem2 in x:
        par2 = np.zeros(x.shape[0])
        par2[np.where(x==elem2)] = 1
        res1 = inference_mandani_2_premissas_and(par1, us_bom, par2, ur_bom, yg_bom)
        res2 = inference_mandani(par1, us_acpt, yg_acpt)
        res3 = inference_mandani_2_premissas_and(par1, us_ruim, par2, ur_ruim, yg_ruim)
        final_result = np.maximum(np.maximum(res1, res2), res3)
        resultado[np.where(x==elem), np.where(x==elem2)]=defuzz(y, final_result)

#ux,uy = np.meshgrid(x,x)
#fig = plt.figure()
#ax = fig.add_subplot(111, projection ='3d')
#ax.plot_wireframe(ux,uy,resultado, rstride=1,cstride = 1)
#ax.set_xlabel("X")
#ax.set_ylabel("Y")
#ax.set_zlabel("Grau de Pertinência")
#fig.show()

fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(8, 9))

ax0.plot(x, us_ruim, 'b', linewidth=1.5, label='Ruim')
ax0.plot(x, us_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax0.plot(x, us_bom, 'r', linewidth=1.5, label='Bom')
ax0.set_title('Service quality')
ax0.legend()

ax1.plot(x, ur_ruim, 'b', linewidth=1.5, label='Ruim')
ax1.plot(x, ur_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax1.plot(x, ur_bom, 'r', linewidth=1.5, label='Bom')
ax1.set_title('Food quality')
ax1.legend()

ax2.plot(y, yg_ruim, 'b', linewidth=1.5, label='Pouco')
ax2.plot(y, yg_acpt, 'g', linewidth=1.5, label='Medio')
ax2.plot(y, yg_bom, 'r', linewidth=1.5, label='Alto')
ax2.set_title('Tip amount')
ax2.legend()

plt.tight_layout()
fig.show()



In [2]:
x = np.arange(0, 10.1, 0.1)
y = np.arange(0, 26, 1)

us_ruim = trapmf(x, [-5, 0, 2, 5])
us_acpt = trapmf(x, [0, 5, 7, 10])
us_bom = trapmf(x, [5, 10, 12, 15])

ur_ruim = trapmf(x, [-5, 0, 2, 5])
ur_acpt = trapmf(x, [2, 5, 6, 8])
ur_bom = trapmf(x, [6, 10, 12, 14])

yg_ruim = trapmf(y, [-12, 0, 2, 12])
yg_acpt = trapmf(y, [0, 12, 14, 25])
yg_bom = trapmf(y, [12, 25, 27, 37])

fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(8, 9))

ax0.plot(x, us_ruim, 'b', linewidth=1.5, label='Ruim')
ax0.plot(x, us_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax0.plot(x, us_bom, 'r', linewidth=1.5, label='Bom')
ax0.set_title('Service quality')
ax0.legend()

ax1.plot(x, ur_ruim, 'b', linewidth=1.5, label='Ruim')
ax1.plot(x, ur_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax1.plot(x, ur_bom, 'r', linewidth=1.5, label='Bom')
ax1.set_title('Food quality')
ax1.legend()

ax2.plot(y, yg_ruim, 'b', linewidth=1.5, label='Pouco')
ax2.plot(y, yg_acpt, 'g', linewidth=1.5, label='Medio')
ax2.plot(y, yg_bom, 'r', linewidth=1.5, label='Alto')
ax2.set_title('Tip amount')
ax2.legend()

plt.tight_layout()
fig.show()

resultado = np.zeros((101,101))
for elem in x:
    par1 = np.zeros(x.shape[0])
    par1[np.where(x==elem)] = 1
    for elem2 in x:
        par2 = np.zeros(x.shape[0])
        par2[np.where(x==elem2)] = 1
        res1 = inference_mandani_2_premissas_and(par1, us_bom, par2, ur_bom, yg_bom)
        res2 = inference_mandani(par1, us_acpt, yg_acpt)
        res3 = inference_mandani_2_premissas_and(par1, us_ruim, par2, ur_ruim, yg_ruim)
        final_result = np.maximum(np.maximum(res1, res2), res3)
        resultado[np.where(x==elem), np.where(x==elem2)]=defuzz(y, final_result)

ux,uy = np.meshgrid(x,x)
fig = plt.figure()
ax = fig.add_subplot(111, projection ='3d')
ax.plot_wireframe(ux,uy,resultado, rstride=1,cstride = 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Gorjeta")
fig.show()


## ITEM 2

C:\Users\matheus_sivelli\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in int_scalars
  del sys.path[0]
C:\Users\matheus_sivelli\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in int_scalars
  if sys.path[0] == '':


In [30]:
x = np.arange(0, 10.1, 0.1)
y = np.arange(0, 26, 1)


us_ruim = trimf(x, [-5, 0, 5])
us_acpt = trimf(x, [0, 5, 10])
us_bom = trimf(x, [5, 10, 15])

ur_ruim = trimf(x, [-5, 0, 5])
ur_acpt = trimf(x, [2, 5, 8])
ur_bom = trimf(x, [6, 10, 14])

yg_ruim = trimf(y, [-12.5, 0, 12.5])
yg_acpt = trimf(y, [0, 12.5, 25])
yg_bom = trimf(y, [12.5, 25, 37.5])

resultado = np.zeros((101,101))
for elem in x:
    par1 = np.zeros(x.shape[0])
    par1[np.where(x==elem)] = 1
    for elem2 in x:
        par2 = np.zeros(x.shape[0])
        par2[np.where(x==elem2)] = 1
        res1 = inference_PROD_2_premissas_and(par1, us_bom, par2, ur_bom, yg_bom)
        res2 = inference_PROD(par1, us_acpt, yg_acpt)
        res3 = inference_PROD_2_premissas_and(par1, us_ruim, par2, ur_ruim, yg_ruim)
        final_result = np.maximum(np.maximum(res1, res2), res3)
        resultado[np.where(x==elem), np.where(x==elem2)]=defuzz(y, final_result)

ux,uy = np.meshgrid(x,x)
fig = plt.figure()
ax = fig.add_subplot(111, projection ='3d')
ax.plot_wireframe(ux,uy,resultado, rstride=1,cstride = 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Gorjeta")
fig.show()

## ITEM 4

In [48]:
x = np.arange(0, 10.1, 0.1)
y = np.arange(0, 26, 1)

us_ruim = trapmf(x, [-5, 0, 2, 5])
us_acpt = trapmf(x, [0, 5, 7, 10])
us_bom = trapmf(x, [5, 10, 12, 15])

ur_ruim = trapmf(x, [-5, 0, 2, 5])
ur_acpt = trapmf(x, [2, 5, 6, 8])
ur_bom = trapmf(x, [6, 10, 12, 14])

yg_ruim = trapmf(y, [-12, 0, 2, 12])
yg_acpt = trapmf(y, [0, 12, 14, 25])
yg_bom = trapmf(y, [12, 25, 27, 37])

fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(8, 9))

ax0.plot(x, us_ruim, 'b', linewidth=1.5, label='Ruim')
ax0.plot(x, us_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax0.plot(x, us_bom, 'r', linewidth=1.5, label='Bom')
ax0.set_title('Service quality')
ax0.legend()

ax1.plot(x, ur_ruim, 'b', linewidth=1.5, label='Ruim')
ax1.plot(x, ur_acpt, 'g', linewidth=1.5, label='Aceitavel')
ax1.plot(x, ur_bom, 'r', linewidth=1.5, label='Bom')
ax1.set_title('Food quality')
ax1.legend()

ax2.plot(y, yg_ruim, 'b', linewidth=1.5, label='Pouco')
ax2.plot(y, yg_acpt, 'g', linewidth=1.5, label='Medio')
ax2.plot(y, yg_bom, 'r', linewidth=1.5, label='Alto')
ax2.set_title('Tip amount')
ax2.legend()

plt.tight_layout()
fig.show()

resultado = np.zeros((101,101))
for elem in x:
    par1 = np.zeros(x.shape[0])
    par1[np.where(x==elem)] = 1
    for elem2 in x:
        par2 = np.zeros(x.shape[0])
        par2[np.where(x==elem2)] = 1
        res1 = inference_PROD_2_premissas_and(par1, us_bom, par2, ur_bom, yg_bom)
        res2 = inference_PROD(par1, us_acpt, yg_acpt)
        res3 = inference_PROD_2_premissas_and(par1, us_ruim, par2, ur_ruim, yg_ruim)
        final_result = np.maximum(np.maximum(res1, res2), res3)
        resultado[np.where(x==elem), np.where(x==elem2)]=defuzz(y, final_result)

ux,uy = np.meshgrid(x,x)
fig = plt.figure()
ax = fig.add_subplot(111, projection ='3d')
ax.plot_wireframe(ux,uy,resultado, rstride=1,cstride = 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Gorjeta")
fig.show()

## ITEM 5

C:\Users\matheus_sivelli\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in int_scalars
  
C:\Users\matheus_sivelli\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in int_scalars
  del sys.path[0]


101
101
101


In [61]:
## ITEM 2

x = np.arange(0, 10.1, 0.1) # Nota de 0 a 10 para as variavéis estudadas
y = np.arange(0, 5.1, 0.1) # Avaliação das estrelas


u_entrega_lenta = trimf(x, [-5, 0, 5])
u_entrega_media = trimf(x, [5, 6, 8])
u_entrega_rapida = trimf(x, [8, 10, 12])

u_atendimento_ruim = trimf(x, [-2, 1, 4])
u_atendimento_medio = trimf(x, [4, 6, 8])
u_atendimento_bom = trimf(x, [8, 10, 12])

y_avaliacao_um_estrela = trimf(y, [-0.7, 0, 0.7])
y_avaliacao_duas_estrela = trimf(y, [0.7, 1.5, 2.2])
y_avaliacao_tres_estrela = trimf(y, [2.2, 2.7, 3.2])
y_avaliacao_quatro_estrela = trimf(y, [3.2, 3.7, 4.2])
y_avaliacao_cinco_estrela = trimf(y, [4.2, 4.8, 5.5])

fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(8, 9))

ax0.plot(x, u_entrega_lenta, 'b', linewidth=1.5, label='Entrega lenta')
ax0.plot(x, u_entrega_media, 'g', linewidth=1.5, label='Entrega Média')
ax0.plot(x, u_entrega_rapida, 'r', linewidth=1.5, label='Entrega Rápida')
ax0.set_title('Qualidade da Entrega')
ax0.legend()

ax1.plot(x, u_atendimento_ruim, 'b', linewidth=1.5, label='Atendimento Ruim')
ax1.plot(x, u_atendimento_medio, 'g', linewidth=1.5, label='Atendimento Médio')
ax1.plot(x, u_atendimento_bom, 'r', linewidth=1.5, label='Atendimento Bom')
ax1.set_title('Qualidade do Atendimento')
ax1.legend()

ax2.plot(y, y_avaliacao_um_estrela, 'b', linewidth=1.5, label='Uma Estrela')
ax2.plot(y, y_avaliacao_duas_estrela, 'g', linewidth=1.5, label='Duas Estrela')
ax2.plot(y, y_avaliacao_tres_estrela, 'r', linewidth=1.5, label='Três Estrela')
ax2.plot(y, y_avaliacao_quatro_estrela, 'b', linewidth=1.5, label='Quatro Estrela')
ax2.plot(y, y_avaliacao_cinco_estrela, 'g', linewidth=1.5, label='Cinco Estrela')
ax2.set_title('Avaliação imposta')
ax2.legend()

plt.tight_layout()
fig.show()

#Atendimento Ruim, Entrega ruim = Uma estrela
#Atendimento Bom, entrega rápida = Cinco Estrela

resultado = np.zeros((101,101))
for elem in x:
    par1 = np.zeros(x.shape[0])
    par1[np.where(x==elem)] = 1
    for elem2 in x:
        par2 = np.zeros(x.shape[0])
        par2[np.where(x==elem2)] = 1
        res1 = inference_mandani_2_premissas_and(par1, u_entrega_lenta, par2, u_atendimento_ruim, y_avaliacao_um_estrela)
        res2 = inference_mandani_2_premissas_and(par1, u_entrega_rapida, par2, u_atendimento_bom, y_avaliacao_cinco_estrela)
        #final_result = np.maximum(res1, res2)
        res3 = inference_mandani_2_premissas_and(par1, u_entrega_rapida, par2, u_atendimento_medio, y_avaliacao_tres_estrela)
        res4 = inference_mandani_2_premissas_and(par1, u_entrega_media, par2, u_atendimento_bom, y_avaliacao_quatro_estrela)
        res5 = inference_mandani_2_premissas_and(par1, u_entrega_media, par2, u_atendimento_medio, y_avaliacao_tres_estrela)
        res6 = inference_mandani_2_premissas_and(par1, u_entrega_lenta, par2, u_atendimento_medio, y_avaliacao_duas_estrela)
        res7 = inference_mandani_2_premissas_and(par1, u_entrega_media, par2, u_atendimento_ruim, y_avaliacao_duas_estrela)
        res8 = inference_mandani_2_premissas_and(par1, u_entrega_lenta, par2, u_atendimento_bom, y_avaliacao_tres_estrela)
        res9 = inference_mandani_2_premissas_and(par1, u_entrega_rapida, par2, u_atendimento_ruim, y_avaliacao_duas_estrela)
        final_result = np.maximum(np.maximum(np.maximum(np.maximum(np.maximum(np.maximum(np.maximum(np.maximum(res1, res2), res3), res4), res5), res6), res7), res8), res9)
        resultado[np.where(x==elem), np.where(x==elem2)]=defuzz(y, final_result)

print(resultado)
ux,uy = np.meshgrid(x,x)
fig = plt.figure()
ax = fig.add_subplot(111, projection ='3d')
ax.plot_wireframe(ux,uy,resultado, rstride=1,cstride = 1)
ax.set_xlabel("Entrega")
ax.set_ylabel("Atendimento")
ax.set_zlabel("Avaliação")
fig.show()



[[0.25282051 0.24951311 0.24664709 ... 2.7        2.7        2.7       ]
 [0.25282051 0.24951311 0.24664709 ... 2.7        2.7        2.7       ]
 [0.25282051 0.24951311 0.24664709 ... 2.7        2.7        2.7       ]
 ...
 [1.46367713 1.46456693 1.46477136 ... 4.70505676 4.70505676 4.70505676]
 [1.46367713 1.46456693 1.46477136 ... 4.70505676 4.70607453 4.70607453]
 [1.46367713 1.46456693 1.46477136 ... 4.70505676 4.70607453 4.70707071]]
